In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Chronic_Fatigue_Syndrome/GSE16059'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene Expression in Peripheral Blood Leucocytes in Monozygotic Twins Discordant for Chronic Fatigue"
!Series_summary	"Background. Chronic fatiguing illness remains a poorly understood syndrome of unknown pathogenesis. We attempted to identify biomarkers for chronic fatiguing illness using microarrays to query the transcriptome in peripheral blood leukocytes. Methods. Cases were 44 individuals who were clinically evaluated and found to meet standard international criteria for chronic fatigue syndrome or idiopathic chronic fatigue, and controls were their monozygotic co-twins who were clinically evaluated and never had even one month of impairing fatigue. Biological sampling conditions were standardized and RNA stabilizing media were used. These methodological features provide rigorous control for bias resulting from case-control mismatched ancestry and experimental error. Individual gene expression profiles were assessed using Affymetrix Human Genom

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Question 1: Gene Expression Data Availability
is_gene_available = True  # Since the dataset uses Affymetrix Human Genome U133 Plus 2.0 arrays

# Question 2.1: Data Availability
trait_row = 2  # Based on 'diagnonsis' values which include 'CFS' and 'ICF'
age_row = None  # Age information is not available in the sample characteristics
gender_row = 1  # Based on 'sex' values which include 'female' and 'male'

# Question 2.3: Data Type Conversion
def convert_trait(value):
    mapping = {'diagnonsis: unaffected': 0, 'diagnonsis: CFS': 1, 'diagnonsis: ICF': 1}
    return mapping.get(value, None)

def convert_age(value):
    # Since age data is not available, define a dummy function
    return None

def convert_gender(value):
    mapping = {'sex: female': 0, 'sex: male': 1}
    return mapping.get(value, None)

# Question 3: Save Metadata
save_cohort_info('GSE16059', './preprocessed/Chronic_Fatigue_Syndrome/cohort_info.json', is_gene_available, trait_row is not None)

# Question 4: Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Chronic_Fatigue_Syndrome', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Chronic_Fatigue_Syndrome/trait_data/GSE16059.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM402241': [0, 0], 'GSM402242': [1, 0], 'GSM402243': [0, 0], 'GSM402244': [1, 0], 'GSM402245': [0, 0], 'GSM402246': [1, 0], 'GSM402247': [0, 1], 'GSM402248': [1, 1], 'GSM402249': [1, 0], 'GSM402250': [0, 0], 'GSM402251': [1, 0], 'GSM402252': [0, 0], 'GSM402253': [1, 0], 'GSM402254': [0, 0], 'GSM402255': [1, 0], 'GSM402256': [0, 0], 'GSM402257': [1, 0], 'GSM402258': [0, 0], 'GSM402259': [1, 0], 'GSM402260': [0, 0], 'GSM402261': [0, 0], 'GSM402262': [1, 0], 'GSM402263': [1, 0], 'GSM402264': [0, 0], 'GSM402265': [0, 1], 'GSM402266': [1, 1], 'GSM402267': [1, 0], 'GSM402268': [0, 0], 'GSM402269': [0, 0], 'GSM402270': [1, 0], 'GSM402271': [0, 0], 'GSM402272': [1, 0], 'GSM402273': [0, 0], 'GSM402274': [1, 0], 'GSM402275': [1, 0], 'GSM402276': [0, 0], 'GSM402277': [1, 0], 'GSM402278': [0, 0], 'GSM402279': [0, 0], 'GSM402280': [1, 0], 'GSM402281': [1, 0], 'GSM402282': [0, 0], 'GSM402283': [1, 0], 'GSM402284': [0, 0], 'GSM402285': [1, 0], 'GSM402286': [0, 0], 'GSM402287': [1, 0], 'GSM402288':

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# Substep 1
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# Substep 2
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# Substep 3
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Chronic_Fatigue_Syndrome/gene_data/GSE16059.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Chronic_Fatigue_Syndrome')

# 4. Save the cohort information.
save_cohort_info('GSE16059', './preprocessed/Chronic_Fatigue_Syndrome/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Chronic_Fatigue_Syndrome/GSE16059.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Chronic_Fatigue_Syndrome', the least common label is '0.0' with 44 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Chronic_Fatigue_Syndrome' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 10 occurrences. This represents 11.36% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ./preprocessed/Chronic_Fatigue_Syndrome/cohort_info.json
